# Variational Auto-Encoder

[Main paper](https://arxiv.org/pdf/1312.6114.pdf)
  
  
In this setting, I have a some data $x$ that is assumed to be come from some distribution that is unkown to us. In the [Ideal World](https://en.wikipedia.org/wiki/Theory_of_forms), the following happens:

  1. A latent variable $z{(i)}$ is sampled from some probability distribution $p(z)$.
  2. The datapoint $x^{(i)}$ is then sampled from $p(x \vert z^{(i)})$
  
We assume that $p(z) = \mathcal N(0, I)$. And we can observe $x^{(i)}$, but we don't know anything else. 

We'd like to learn $p(x \vert z)$ so we can sample new points from $p(z)$ and generate new data points $x$. If we had rows of $<z^{(i)}, x^{(i)}>$, then it boils down to the well-understood regression or classification problem, where given $z$ we can learn $p(x \vert z)$. But we don't have those rows. So what do we do?

If we can generate $z$ from $x$ somehow, it gives us the trainset to learn $p(x \vert z)$. But in order to generate $z$ from $x$ we need to learn $p(z \vert x)$. Lets say we learn a distribution $q(z \vert x)$ that approximates $p(z \vert x)$. The per instance loss function that this learning algorithm would need to minimize is -

$$
L^{(i)} = KL\left( q(z \vert x^{(i)}) \;\vert \vert\; p(z \vert x^{(i)}) \right)
$$

After some [algebraic manipulation](https://youtu.be/uaaqyVS9-rM?t=19m42s), this boils down to minimizing the following per-instance loss:

$$
\mathcal L^{(i)} = -\mathbb E_{q(z \vert x^{(i)})}[log p(x^{(i)} \vert z)] + KL(q(z \vert x^{(i)}) \;\vert\vert\; p(z))
$$

Another simplifying assumption is that $q(z \vert x) = \mathcal N(z; \mu(x), \sigma(x))$. Where the parameters of the distribution $\mu$ and $\sigma$ depend on $x$.

So given a pariticular value of $x = x^{(i)}$, $q(z \vert x^{(i)}) = \mathcal N(z; \mu(x^{(i)}), \sigma(x^{(i)}))$

Learning $q(z \vert x)$ then boils down to learning $\mu$ and $\sigma$. We can create a DNN for this, often called the **encoder network**, that takes in a datapoint $x^{(i)}$ and outputs $\mu$ and $\sigma$ that can then be used as parameters for $q(z \vert x)$. We then sample a $z^{(i)}$ from this distribution and pass it through another DNN, often called the **decoder network** or the **regeneration network** which outputs the $p(x^{(i)} \vert z^{(i)})$. Both these DNNs can be trained simultaneously by minimizing the aggregate loss function $\mathcal L^{(i)}$ across each mini-batch.

Lets look at both the terms of the loss function. The first term is an expectation even though this is for a single observation $x^{(i)}$. This can be thought of as follows - when sampling from $q(z \vert x^{(i)})$, don't just pick out a single datapoint $z^{(i)}$, but rather pick out $L$ datapoints $z^{(i, l)}$. This is to say that all of these $L$ points will result in the generation of the datapoint $x^{(i)}$. The expectation is then approximated as the sum over all these L samples.

$$
-\mathbb E_{q(z \vert x^{(i)})}[log p(x^{(i)} \vert z)] \simeq  -\frac1L \sum_{l=1}^L log\;p(x^{(i)} \vert z^{(i, l)})
$$

According to Kingma in the main paper if the mini-batch size is big enough, say 100, it is ok to take $L=1$. In most of the implementations that I have seen so far, $L=1$ even for smaller mini-batches. The above expectation now becomes -

$$
-\mathbb E_{q(z \vert x^{(i)})}[log p(x^{(i)} \vert z)] \simeq -log\;p(x^{(i)} \vert z^{(i)})
$$

This is nothing but the negative log-likelihood loss. For regression problems this is the MSE, for binary classification this is BCE. See the Recap section below to see why.

Based on the formula of the [KL-Divergence of 2 gaussians](https://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians), if we just plugin in the values $\mu^{(i)}$, $\sigma^{(i)}$, $0$, and $1$ we get -

$$
KL(q(z \vert x)\;\vert\vert\;p(z)) = -\frac12 \sum_{j=1}^D \left(1 + log((\sigma_j^{(i)})^2) - (\mu^{(i)})^2 - (\sigma_j^{(i)})^2 \right)
$$

Here $\sigma^{(i)}$ and $\mu^{(i)}$ are the outputs of the encoder network for each input $x^{(i)}$ and $D$ is the dimension of the $\mu$ and $\sigma$ vectors. What the above expression is doing is first calculating the inner term element-wise, this will give a vector of size $D$. The elements of this resulting vector are then summed up to give the KL-Divergence.

So the final instance-wise loss function is give by -

$$
\mathcal L^{(i)} = -log\;p(x^{(i)} \vert z^{(i)}) - \frac12 \sum_{j=1}^D \left(1 + log((\sigma_j^{(i)})^2) - (\mu^{(i)})^2 - (\sigma_j{(i)})^2 \right)
$$

And the batch loss is - as usual - given by aggregating the instance-wise loss function -

$$
J = \sum_{i=1}^m \mathcal L^{(i)}
$$

The [pytorch implementation](https://github.com/pytorch/examples/blob/master/vae/main.py) takes the sum of the instance-wise loss, [Keras](https://blog.keras.io/building-autoencoders-in-keras.html) example takes the mean.

Once the decoder network is trained, i.e., we know $p(x \vert z)$, remember that was our main goal here, we can then sample points from a standard normal distribution, which is what $p(z)$ is assumed to be, and generate datapoints that are similar to our training set.


## References

  * https://www.jeremyjordan.me/variational-autoencoders/
  * https://www.youtube.com/watch?v=R3DNKE3zKFk
  * https://www.youtube.com/watch?v=5WoItGTWV54&feature=youtu.be&t=26m32s
  * https://www.youtube.com/watch?v=9zKuYvjFFS8
  * https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf

## TODO

 1. Play with the pytorch example and verify that I can generate real-looking MNIST digits from samples in a standard normal distribution. It just sounds too much like black magic! I need to try it out to convince myself it works.
 
 2. Make $\mu$ and $\sigma$ 2D, which means that $z^{(i)}$ will also be 2D. Then visualize $z$s color coding them by classes of $x$s. I want to verify that even though I am generating $\mu$ and $\sigma$ for each $x^{(i)}$, the $z^{(i)}$s generated by the same classes should be more-or-losses clustered together.
 
 3. Continuing the above experiment, overlay all the color coded $z^{(i)}$s on a standard normal distribution - which is supposed to be the prior $p(z)$. This will further strengthen my intuition that it is indeed possible to generate new datapoints just from the standard normal distribution.

### Recap
Any ML problem can be thought of as a two step process - first define a parameterized p.m.f or p.d.f $p_w(y \vert x)$, and then learn the parameters of this function. For binary classification problem $p_w(y \vert x) = \sigma(w^Tx + b)$ where $\sigma$ is the sigmoid function. For regression problem, $f_w(y \vert x) = \mathcal N(y; w^Tx+b, \sigma(x)^2)$ where $\sigma^2$ is the variance of the Normal distribution. Strictly speaking we don't learn all the parameters in a regression problem, we only learn $\mu = w^Tx + b$ but we don't learn $\sigma(x)^2$. However, it is good enough to give us an estimate of $y$.

When using a DNN, $p_w(y \vert x)$ is given by a some complex function based on the NN architecture, and we learn the parameters $w$. But the main concept remains the same, i.e., first define what the function will look like (encoded in the NN architecture), and then learn the parameters of the function. 